In [139]:
import warnings
warnings.filterwarnings("ignore")

In [351]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import pandas as pd

# 데이터 불러오기
df = pd.read_csv('거리계산(중복제거)/DB_변수추가/분석용데이터_전처리완/연립다세대_월세_전처리7-5.csv')#, encoding='euc-kr'
#df.dropna(inplace=True)

# 년도와 분기 열을 이용하여 년도_분기 변수 추가
df['년도_분기'] = df['년도'].astype(str) + '_' + df['분기'].astype(str)

# 필터링할 행 선택
filter_condition = (df['년도_분기'] == '2019_1') | (df['년도_분기'] == '2020_2') | (df['년도_분기'] == '2021_4') | (df['년도_분기'] == '2022_4')
df_filtered = df[filter_condition]
df_filtered = df
# 독립 변수와 종속 변수 분리
X = df_filtered[['월세금(만원)','건축년도', '전용면적(㎡)', '층', '편의점(개)', '약국(개)', '대형마트(개)', '경찰서(개)',
        '소방서(개)', '공공도서관(개)', '병원(개)', '초등학교 거리(m)', 
        '중학교 거리(m)', '고등학교 거리(m)']]
y = df_filtered['보증금(만원)']

# 훈련 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBoost 모델 설정
xgb_model = xgb.XGBRegressor(objective='reg:squarederror')

# 모델 훈련
xgb_model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = xgb_model.predict(X_test)

# 모델 성능 평가 (결정계수)
r_squared = r2_score(y_test, y_pred)
print("XGBoost R^2 Score:", r_squared)

# 독립 변수의 중요도 출력
feature_importance = pd.DataFrame({'Variable': X.columns, 'Importance': xgb_model.feature_importances_})
top_5_features = feature_importance.sort_values(by='Importance', ascending=False).head(5)
print("Top 5 XGBoost Feature Importances:")
print(top_5_features)

XGBoost R^2 Score: 0.6137375634635819
Top 5 XGBoost Feature Importances:
   Variable  Importance
0   월세금(만원)    0.364958
2   전용면적(㎡)    0.127060
1      건축년도    0.081361
3         층    0.065269
9  공공도서관(개)    0.062281


In [363]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import pandas as pd

# 데이터 불러오기
df = pd.read_csv('거리계산(중복제거)/DB_변수추가/분석용데이터_전처리완/연립다세대_월세_XGB분석.csv')

# 년도와 분기 열을 이용하여 년도_분기 변수 추가
df['년도_분기'] = df['년도'].astype(str) + '_' + df['분기'].astype(str)

# 필터링할 년도_분기 선택
filtered_quarters = ['2019_1', '2020_2', '2021_4', '2022_4']
results = {}

for quarter in filtered_quarters:
    # 해당 분기의 데이터만 필터링
    df_filtered = df[df['년도_분기'] == quarter]

    # 독립 변수와 종속 변수 분리
    X = df_filtered[['월세금(만원)','건축년도', '전용면적(㎡)', '층', '편의점(개)', '약국(개)', '대형마트(개)', '경찰서(개)',
            '소방서(개)', '공공도서관(개)', '병원(개)', '초등학교 거리(m)', 
            '중학교 거리(m)', '고등학교 거리(m)']]
    y = df_filtered['보증금(만원)']

    # 훈련 데이터와 테스트 데이터로 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # XGBoost 모델 설정
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror')

    # 모델 훈련
    xgb_model.fit(X_train, y_train)

    # 테스트 데이터에 대한 예측
    y_pred = xgb_model.predict(X_test)

    # 모델 성능 평가 (결정계수)
    r_squared = r2_score(y_test, y_pred)

    # 독립 변수의 중요도 계산
    feature_importance = pd.DataFrame({'Variable': X.columns, 'Importance': xgb_model.feature_importances_})
    top_7_features = feature_importance.sort_values(by='Importance', ascending=False).head(7)

    # 결과 저장
    results[quarter] = {'Top 7 Features': top_7_features, 'R^2 Score': r_squared}

# 결과 출력
for quarter, result in results.items():
    print(f"Quarter: {quarter}")
    print("Top 7 XGBoost Feature Importances:")
    print(result['Top 7 Features'])
    print("R^2 Score:", result['R^2 Score'])
    print()


Quarter: 2019_1
Top 7 XGBoost Feature Importances:
    Variable  Importance
0    월세금(만원)    0.183484
2    전용면적(㎡)    0.133161
9   공공도서관(개)    0.087568
1       건축년도    0.084225
10     병원(개)    0.079786
3          층    0.062679
6    대형마트(개)    0.054836
R^2 Score: 0.48341092247537754

Quarter: 2020_2
Top 7 XGBoost Feature Importances:
    Variable  Importance
0    월세금(만원)    0.168215
2    전용면적(㎡)    0.109773
1       건축년도    0.096564
10     병원(개)    0.091649
9   공공도서관(개)    0.067148
6    대형마트(개)    0.063979
3          층    0.060286
R^2 Score: 0.4684791210178971

Quarter: 2021_4
Top 7 XGBoost Feature Importances:
    Variable  Importance
0    월세금(만원)    0.181177
2    전용면적(㎡)    0.137473
1       건축년도    0.092616
10     병원(개)    0.080133
9   공공도서관(개)    0.070527
3          층    0.058749
8     소방서(개)    0.057909
R^2 Score: 0.5357946645446481

Quarter: 2022_4
Top 7 XGBoost Feature Importances:
    Variable  Importance
0    월세금(만원)    0.241160
2    전용면적(㎡)    0.126368
1       건축년도    0.095718
9 